In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
import pandas as pd
import plotly.express as px

In [3]:
from app.utils import t20_matches

ModuleNotFoundError: No module named 'app'

In [ ]:
from notebook_utils import Statter

In [ ]:
matches = t20_matches()
print("loaded", len(matches), "matches")

In [ ]:
stats = []

over_tots = defaultdict(Statter)
for m in matches:
    for inns in m.innings:
        for over in inns.overs:
            over_tots[over.over].add(over.runs)
stats = [{"over": k, "division": "all matches", "avg runs": v.mean} for k, v in over_tots.items()]

over_tots = defaultdict(Statter)
for m in matches:
    if m.info.gender == "male":
        for inns in m.innings:
            for over in inns.overs:
                over_tots[over.over].add(over.runs)
stats.extend({"over": k, "division": "males", "avg runs": v.mean} for k, v in over_tots.items())

over_tots = defaultdict(Statter)
for m in matches:
    if m.info.gender == "female":
        for inns in m.innings:
            for over in inns.overs:
                over_tots[over.over].add(over.runs)
stats.extend({"over": k, "division": "females", "avg runs": v.mean} for k, v in over_tots.items())

In [ ]:
df = pd.DataFrame(stats)

In [ ]:
fig = px.scatter(df, x="over", y="avg runs", color="division")
fig.show()

In [ ]:
fig=px.scatter(df,x="over", y="avg runs", facet_col="division")
fig.show()

In [ ]:
def collect_over_stats_by_year(yy: int) -> list[dict]:
    over_tots = defaultdict(Statter)
    for m in matches:
        if m.info.dates[0].year == yy and m.info.gender=="male":
            for inns in m.innings:
                for over in inns.overs:
                    over_tots[over.over].add(over.runs)
    return [{"over": k, "year": yy, "avg runs": v.mean} for k, v in over_tots.items()]

stats=[]
for yr in range(2015,2025,2):
    stats.extend(collect_over_stats_by_year(yr))
df = pd.DataFrame(stats)
df.head()
fig = px.scatter(df, x="over", y="avg runs", color="year")
fig.show()

In [ ]:
from typing import Callable
from app.ingest.classes import Match
def over_stats(matches,filter_func: Callable,filter_value, filtered_field: str) -> list[dict]:
    over_tots = defaultdict(Statter)
    for m in matches:
        if filter_func(m) == filter_value:
            for inns in m.innings:
                for over in inns.overs:
                    over_tots[over.over].add(over.runs)

    return [{"over": k, filtered_field: filter_value, "avg runs": v.mean} for k, v in over_tots.items()]

In [ ]:
males = [m for m in matches if m.info.gender == "male"]
stats = []
for yr in range(2010, 2024, 2):
    stats.extend(over_stats(males, lambda m: m.info.dates[0].year, yr, "year"))
df = pd.DataFrame(stats)
fig = px.scatter(df, x="over", y="avg runs", color="year")
fig.show()